In [57]:
from table import HoldemTable, OmahaTable
import hand


from exceptions import *
from utils import *
from hand import Hand
from ranker import *

In [84]:
num = ["2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K", "A"]
suit = ["d", "c", "s", "h"]
a = [n + s for n in num for s in suit]
hand_ranking_order = ["High Card", "One Pair", "Two Pairs", "Three of a Kind", "Straight", "Flush", "Full House", "Four of a Kind", "Straight Flush", "Royal Flush"]


In [85]:

def give_odds(flop, turn, river, p1_hand, p2_hand):
    #flop cards and calcs
    game = HoldemTable(num_players= 2, deck_type='full')
    game.add_to_hand(1, p1_hand)
    game.add_to_hand(2, p2_hand)

    game.add_to_community(flop)
    community_cards, undrawn_combos = game.simulation_preparation(150000)
    flop_strength = game.hand_strength_analysis(res_arr = game.simulate_calculation(community_cards, undrawn_combos))
    
    #turn cards and calcs
    game.add_to_community(turn)
    community_cards, undrawn_combos = game.simulation_preparation(150000)
    turn_strength = game.hand_strength_analysis(res_arr = game.simulate_calculation(community_cards, undrawn_combos))

    #river cards and calcs
    game.add_to_community(river)
    who_won, what_hand = winner(game)
    return flop_strength, turn_strength, who_won, what_hand

def winner(self):
    player_rank = np.zeros(self.num_players, dtype=np.int64)
    player_hand_type = np.zeros(self.num_players, dtype=np.int64)

    for player in range(self.num_players):
        player_combos, player_res_arr = self.player_hands[player + 1].hand_value(self.community_arr)
        player_rank[player] = np.max(player_res_arr)
        player_hand_type[player] = np.max(player_res_arr) // 16 ** 5

    if (np.max(player_rank) == player_rank).sum() == 1:
        if np.argmax(player_rank) + 1 == 1:
            return np.argmax(player_rank) + 1, hand_type_dict[player_hand_type[np.argmax(player_rank)]]
        else:
            return 0, hand_type_dict[player_hand_type[np.argmax(player_rank)]]
    else:
        winners, = np.where(np.max(player_rank) == player_rank)
        return 0.5, hand_type_dict[player_hand_type[winners[0]]]

In [86]:


def find_vectors(flop_odds, turn_odds, winning_player, winning_hand):
    flop_vector = [0] * len(hand_ranking_order)
    for hand, probability in flop_odds[1].items():
        if hand in hand_ranking_order:
            index = hand_ranking_order.index(hand)
            flop_vector[index] = probability 

    #Initialize odds vector with 0 for all possible hands
    odds_vector = [0] * len(hand_ranking_order)

    #Assuming turn_odds[1] is a dictionary mapping hand ranking names to their odds
    for hand, probability in turn_odds[1].items():
        if hand in hand_ranking_order:
            index = hand_ranking_order.index(hand)
            odds_vector[index] = probability 

    #winning player vector, with index of winning hand set to 1
    win_vector = [0] * (len(hand_ranking_order) + 1)
    if winning_hand in hand_ranking_order:
        index = hand_ranking_order.index(hand)
        win_vector[index] = 1
    #last column gives 1 if player 1 wins, 0.5 if tie, and 0 if player 2 wins
    win_vector[-1] = winning_player
    return flop_vector, odds_vector, win_vector



In [87]:
#Example usage
p1hand = ['Td', 'Ts']
p2hand = ['Ad', 'As']
flop = ['2d', '3d', '4d']
turn = ['5d']
river = ['6d']
flop_odds, turn_odds, winning_player, winning_hand = give_odds(flop, turn, river, p1hand, p2hand)
flop_final, turn_final, win_final = find_vectors(flop_odds, turn_odds, winning_player, winning_hand)
print("Flop Vector: ", flop_final)
print("Turn Vector: ", turn_final)
print("Win Vector: ", win_final)

Flop Vector:  [0, np.float64(28.79), np.float64(28.89), np.float64(5.25), np.float64(1.52), np.float64(32.63), np.float64(2.73), np.float64(0.1), np.float64(0.1), 0]
Turn Vector:  [0, 0, 0, 0, 0, np.float64(97.73), 0, 0, np.float64(2.27), 0]
Win Vector:  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0.5]


Problems: 
 - VERY IMPORTANT: there is an issue with recognizing higher hands within each bracket, i.e., king high flush vs ace high flush. There is a reweighing formula, however would be difficult to implement
    - Below is Chat's response
 - Trim this whole things down to be more computationally effective
 - compute every single permutation, roughly 4.5 trillion 
 - how to get into csv

Proposed Solution:
 - Rough numerical approximations

In [39]:
def get_hand_weight(hand_type: str, highest_card: int) -> float:
    """
    Calculate the weight of a poker hand based on its type and highest card.

    Parameters:
    hand_type (str): The type of poker hand (e.g., "Royal Flush", "Straight Flush", etc.).
    highest_card (int): The highest card in the hand (2-14, where 14 represents an Ace).

    Returns:
    float: The weighted strength of the poker hand.
    """
    base_weights = {
        "Royal Flush": 1.00,
        "Straight Flush": 0.95,
        "Four of a Kind": 0.90,
        "Full House": 0.80,
        "Flush": 0.70,
        "Straight": 0.60,
        "Three of a Kind": 0.50,
        "Two Pair": 0.35,
        "One Pair": 0.20,
        "High Card": 0.10,
    }

    if hand_type not in base_weights:
        raise ValueError("Invalid hand type")

    base_weight = base_weights[hand_type]
    rank_factor = 0.8 + 0.2 * (highest_card / 14)

    return round(base_weight * rank_factor, 3)

#Example usage:
print(get_hand_weight("Straight Flush", 5))   # Lower Straight Flush
print(get_hand_weight("Straight Flush", 13))  # Higher Straight Flush 

0.828
0.936


In [103]:
def find_value(game):
    community_cards, undrawn_combos = game.simulation_preparation(150000)
    strengh = game.hand_strength_analysis(res_arr = game.simulate_calculation(community_cards, undrawn_combos))
    return strengh[1]    

In [104]:
p1_hand = ['Td', 'Ts']
p2_hand = ['Ad', 'As']
flopgame = ['2d', '3d', '4d']
game = HoldemTable(num_players= 2, deck_type='full')
game.add_to_hand(1, p1_hand)
game.add_to_hand(2, p2_hand)


In [105]:
a = (find_value(game))
print(a)


{np.str_('One Pair'): np.float64(34.58), np.str_('Two Pairs'): np.float64(39.27), np.str_('Three of a Kind'): np.float64(12.07), np.str_('Straight'): np.float64(2.5), np.str_('Flush'): np.float64(1.56), np.str_('Full House'): np.float64(9.09), np.str_('Four of a Kind'): np.float64(0.91), np.str_('Straight Flush'): np.float64(0.02)}
